<a href="https://colab.research.google.com/github/sergeymasl/pandas_cource/blob/main/Data_Types_and_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Types and Missing Values**

## **Введение**

В этом уроке вы изучете какие типы данных есть в DataFrame и Series.
Также вы узнаете как заменять данные в DataFrame и Series.

Для демонстрации мы будем использовать датасет с данными о винах.

In [1]:
# импорт библиотеки
import pandas as pd
# загрузка датасета для дальнейшей работы
reviews = pd.read_csv("https://drive.google.com/uc?export=download&id=1z-1idT4mGbOvHgmEPzneqV54EJ1-w7tk", index_col=0)


## **Типы данных**

>**Типы данных** -  это очень важная вещь, на которую **стоит потратить время** в самом начале вашей работы, то есть при загрузки данных.

>В противном случае, вы можете столкнуться с неправильными результатами или ошибками слишком поздно и на исправление этих ошибок уйдет много-много больше времени и нервов, чем вы бы потратили на работу с типами данных при загрузке.

Есть некоторая разница между типами данных в Pandas, NumPy и Python. Вот сравнительная табличка:

|Pandas        | Python        | NumPy                                                         | Описание                                         |
|---           |---            |---                                                            |---                                                    |
|object        |str или смесь  |string_, unicode_, смешанные типы                              | Текстовые или смешанные числовые и нечисловые значения|
|int64         |int            |int_, int8, int16, int32, int64, uint8, uint16, uint32, uint64 | Целые числа                                           |
|float64       |float          |float_, float16, float32, float64                              | Числа с плавающей точкой                              |
|bool          |bool           |bool_                                                          | Значения True/False                                   |
|datetime64    |datetime             |datetime64[ns]                                                 | Значения даты и времени                               |
|timedelta[ns] |NA             |NA                                                             | Разность между двумя datetimes                        |
|category      |NA             |NA                                                             | Ограниченный список текстовых значений                |

Рассмотрим на примере что может быть не так с данными, возьмем пример с "грязными" данными о вине:

In [11]:
dirty_data = pd.read_csv("https://drive.google.com/uc?export=download&id=11xDK6SRQcClzHHXeLC6rUlc2fmNyFt9i", index_col=0)
dirty_data

,points,price,title,was_tested,sizes_of_bottles,month,day,year
0,87.0,$24.00,Nicosia 2013 Vulkà Bianco (Etna),No,not known,1,10,2021
1,87.0,$15.00,Quinta dos Avidagos 2011 Avidagos Red (Douro),Yes,187,6,15,2021
2,87.0,$14.00,Rainstorm 2013 Pinot Gris (Willamette Valley),Yes,187,3,29,2021
3,87.0,$13.00,St. Julian 2013 Reserve Late Harvest Riesling ...,Yes,375,10,27,2021
4,87.0,$65.00,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Yes,750,2,2,2021


Данные не выглядят "грязно" в прямом смыле это слова, но для корректной работы с ними, необходимо провести некоторые манипуляции.

Попробуем найти сумму всех цен:

In [12]:
dirty_data['price'].sum()

'$24.00$15.00$14.00$13.00$65.00'

В итоге мы получили не арифметическое сложение, а строковую конкатенацию, т.к. столбец ```price``` был загружен в строковом формате из-за символа ```$```.

Проверить какие форматы у столбцов можно с помощью метода ```dtypes``` для DataFrame и ```dtype``` для Series.

In [13]:
dirty_data.dtypes

points              float64
price                object
title                object
was_tested           object
sizes_of_bottles     object
month                 int64
day                   int64
year                  int64
dtype: object

Также можно использовать метод ```info()```

In [14]:
dirty_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   points            5 non-null      float64
 1   price             5 non-null      object 
 2   title             5 non-null      object 
 3   was_tested        5 non-null      object 
 4   sizes_of_bottles  5 non-null      object 
 5   month             5 non-null      int64  
 6   day               5 non-null      int64  
 7   year              5 non-null      int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 360.0+ bytes


После проверки автоматически подгружаемых типов данных, мы можем изменить для удобства дальнейшей работы:

1. `points` - `float64`, ничего страшного в числах с плаващей запятой нет, но при целочисленных значениях удобнее работать с `int64`.
2. `price` - `object`, то есть строковый формат, а нужен числовой.
3. `title` - все ок
4. `was_tested` - `object`, т.к. он содержит два значения `да` и `нет`, то удобнее было бы работать с типом данных `bool` (`True`,`False`)
5. `sizes_of_bottles` - `object` (то есть строковый), этот столбец отвечает за размер бутылки, логично, что он долже быть числовым.
6. Также у нас есть столбцы `Month`, `Day` и `Year`, с которыми удобнее будет работать в `datetime64`.

Для дальнейшей работы, необходимо изменить типы данных:
>`points`: `float64` → `int64`

>`price`: `object` → `float64`

>`was_tested`: `object` → `bool`

>`sizes_of_bottles`: `object` → `int64`

>`Month`, `Day`, `Year`  → `datetime64`

Есть несколько способов преобразования данных в DataFrame:
>Использование метода `astype()`

>Написание собственной функции для сложных изменений

>Использование функций `pd.to_numeric()` и `pd.to_datetime()`

## Использование `astype()`

Первый метод который мы рассмотрим это метод `astype()`
Этот метод подходит при изменении *заменимых* типов данных, то есть для такой замены при которой не потеряется и не исказиться первоначальный смысл данных.

То есть, его можно использовать при изменении целочисленное значение в число с плавающей запятой.
Можно изменять любой тип данных в строковый.

Можно использовать для замены числа с плавающей запятой в целочисленное значение, **НО**, только если вы уверены в том, что вы не потеряете дробные части.

В нашем случае столбец `points` не имеет дробной части, поэтому изменим тип данных на `int64`:

In [25]:
dirty_data['points'] = dirty_data['points'].astype('int')
dirty_data

,points,price,title,was_tested,sizes_of_bottles,month,day,year
0,87,$24.00,Nicosia 2013 Vulkà Bianco (Etna),No,not known,1,10,2021
1,87,$15.00,Quinta dos Avidagos 2011 Avidagos Red (Douro),Yes,187,6,15,2021
2,87,$14.00,Rainstorm 2013 Pinot Gris (Willamette Valley),Yes,187,3,29,2021
3,87,$13.00,St. Julian 2013 Reserve Late Harvest Riesling ...,Yes,375,10,27,2021
4,87,$65.00,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Yes,750,2,2,2021


Если в строковом выражении есть символы которые нельзя интерпритировать как часть числа, то применить метод `astype` не получиться.

In [36]:
# это приобразование вызовет исключение
try:
  dirty_data['price'].astype('float')
# отработаем его и посмотрим на его суть
except Exception as ex:
  print(f'Тип исключения: \033[1m{type(ex)}\033[0m')
  print(f'Описание исключения: \033[1m{ex}\033[0m')

Тип исключения: <class 'ValueError'>
Описание исключения: could not convert string to float: '$24.00'


In [37]:
# попробуем со столбцом sizes_of_bottles
try:
  dirty_data['sizes_of_bottles'].astype('int')
# отработаем исключение его и посмотрим на его суть
except Exception as ex:
  print(f'Тип исключения: \033[1m{type(ex)}\033[0m')
  print(f'Описание исключения: \033[1m{ex}\033[0m')

Тип исключения: <class 'ValueError'>
Описание исключения: invalid literal for int() with base 10: 'not known'


Теперь попробуем применить метод `astype()` к столбцу `was_tested` для преобразования его в булевы величины, и сравним его с первоначальным:

In [ ]:
# соединим два столбца, оригинальный и измененный
pd.concat([dirty_data['was_tested'], dirty_data['was_tested'].astype('bool')], axis = 1)